# Boilerplate functions

In [1]:
import cv2

In [2]:
import tensorflow as tf
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

with tf.Session() as sess:
    print (sess.run(c))

[[22. 28.]
 [49. 64.]]


In [3]:
from scipy.io import loadmat
import numpy as np
import re
import numpy as np
import time
import math
import pandas as pd
import os

import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from matplotlib.ticker import MaxNLocator

import tensorflow as tf
import cv2

from sklearn.model_selection import train_test_split
from keras.preprocessing import image

from keras.preprocessing import image   # for preprocessing the images
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images
from glob import glob
from tqdm import tqdm

import pickle
%matplotlib auto

Using TensorFlow backend.


Using matplotlib backend: Qt5Agg


## Function to read the video frame by frame and store a few frames

In [ ]:
def frame_capture(filename):
    cap = cv2.VideoCapture(filename)
    count = 0
    frame_prefix = filename.split('/')[6]+"_"\
                  +filename.split('/')[7]+"_"\
                  +filename.split('/')[8]+"_"\
                  +(filename.split('/')[9]).split('.')[0]
    
    frameRate = cap.get(5)
    while(cap.isOpened()):
        frameId = cap.get(1)
        ret, frame = cap.read()

        if ret == True:
            # Display the resulting frame : cv2.imshow('Frame',frame)
            
            if (frameId % math.floor(frameRate) == 0):
                framename = '/media/mynewdrive/data/Retail_AI/3DPeS/'+frame_prefix+"_frame%d.jpg" % count
                if not cv2.imwrite(framename, frame):
                    raise Exception("Could not write image")

            # Press Q on keyboard to  exit
            count+=1
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        # Break the loop
        else: 
            break

    # When everything done, release the video capture object
    cap.release()

    # Closes all the frames
    cv2.destroyAllWindows()

In [ ]:
import os
my_dir = '/media/mynewdrive/data/Retail_AI/3DPeS/Video_Set_1/'
for dir_2 in os.listdir(my_dir):
    dir_3 = os.path.join(my_dir,dir_2)
    for dir_4 in os.listdir(dir_3):
        dir_5 = os.path.join(dir_3,dir_4)
        for dir_6 in os.listdir(dir_5):
            filename = os.path.join(dir_5,dir_6)
            frame_capture(filename)                  

# Load YoloV3

In [4]:
#load yol
net = cv2.dnn.readNet("yolov3.weights.1","darknet/cfg/yolov3.cfg")
classes = []
with open("coco.names","r") as f:
    classes = [line.strip() for line in f.readlines()]
                   
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))
    

## Function to detect object in one frame

In [78]:
def detect_frame (img_name):
    img = cv2.imread(img_name)
    img = cv2.resize(img, None, fx=0.34, fy=0.4)
    height, width, channels = img.shape
    
    #detect objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416,416),(0,0,0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    
    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                #Object detected
                center_x = int(detection[0]*width)
                center_y = int(detection[1]*width)
                cv2.circle(img,(center_x,center_y),10,(0,255,0),2)
                w = int(detection[2] * width)
                h = int(detection[3] * width)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.4, 0.6)

    yolo_img_name = img_name.split('/')[0]+"_YOLO/"+(img_name.split('/')[1]).split('.')[0]+"_yolo.jpg"
    cv2.imwrite(yolo_img_name,img)
#     cv2.imshow("Image", img)
#     cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
dir_name = 'Video_Set_1_Frames'
count = 0
for frames in os.listdir(dir_name):
    count += 1
    print(count)
    read_detect_show(dir_name+"/"+frames)
    

## Function to detect and bound objects in a video and store output

In [45]:
def detect_video(file_name,label = 'person'):
    print("Current file :",file_name)
    cap = cv2.VideoCapture(file_name)
    frame_count = 0
    img_array = []
    metadata_file = file_name.split('.')[0]+'.txt'
    video_output = file_name.split('.')[0]+'_yolo.avi'
    label_index = classes.index(label)
    outF = open(metadata_file, "w")
    while True:
        _,frame = cap.read()
        frame_count+=1
        if(_ != True):
            break
            
        height, width, channels = frame.shape
        blob = cv2.dnn.blobFromImage(frame, 0.00392, (416,416), (0,0,0), True, crop = False)
        net.setInput(blob)
        outs = net.forward(output_layers)
        
        class_ids = []
        confidences = []
        boxes = []
        
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5 and class_id == label_index:
                    #Object detected
                    center_x = int(detection[0]*width)
                    center_y = int(detection[1]*height)
                    
                    #draw circle
                    cv2.circle(frame,(center_x,center_y), radius = 5, color = (0,255,0), thickness = 1)
                    w = int(detection[2] * width)
                    h = int(detection[3] * width)
                    
                     # Rectangle coordinates
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)
        
        
        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.4, 0.6)
        font = cv2.FONT_HERSHEY_PLAIN          
    
        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                color = colors[i]
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                cv2.putText(frame, label, (x, y + 30), font, 1, color, 2)

                #create a text file to output bounding boxes
                # write frame number and coordinates
                outF.write("Frame# %f : %f %f %f %f\n" % (frame_count,x,y,w,h))

                    
        img_array.append(frame)
        #cv2.imshow("Image",frame)
        key = cv2.waitKey(1)
        if key == 27:
            break
    cap.release()
    cv2.destroyAllWindows()
    outF.close()   


    #Create the output video file
    size = (width,height)
    out = cv2.VideoWriter(video_output,cv2.VideoWriter_fourcc(*'DIVX'), 15, size)

    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()



### Function to get all the video files in the directory

In [46]:
def getListOfFiles(dirName):
    # create a list of files and sub directories
    #names in the given directory
    listOfFile = os.listdir(dirName)
    allFiles = list()
    listOfFile.sort()
    #iterate over all the entries
    for entry in listOfFile:
        #create full path
        fullPath = os.path.join(dirName, entry)
        #If entry is a directory then get the list of files in this directory
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
    return allFiles

In [48]:
import time
file_list = getListOfFiles('Video_Set_1/')
first_start_time = time.time()
for video_file in file_list:
    file_start_time = time.time()
    detect_video(video_file)
    print("--- %s seconds ---" %(time.time() - start_time))
print("--- %s seconds ---" %(time.time() - first_start_time))

Current file : Video_Set_1/ID_01/Camera_1/Seq_1.avi
--- 106.28248047828674 seconds ---
Current file : Video_Set_1/ID_01/Camera_1/Seq_2.avi
--- 152.86348056793213 seconds ---
Current file : Video_Set_1/ID_01/Camera_2/Seq_1.avi
--- 167.97248768806458 seconds ---
Current file : Video_Set_1/ID_01/Camera_2/Seq_1.txt
--- 171.86449551582336 seconds ---
Current file : Video_Set_1/ID_01/Camera_2/Seq_2.avi
--- 186.62309980392456 seconds ---
Current file : Video_Set_1/ID_01/Camera_3/Seq_1.avi
--- 277.2238459587097 seconds ---
Current file : Video_Set_1/ID_01/Camera_3/Seq_2.avi
--- 302.83773016929626 seconds ---
Current file : Video_Set_1/ID_02/Camera_1/Seq_1.avi
--- 354.9105408191681 seconds ---
Current file : Video_Set_1/ID_02/Camera_1/Seq_2.avi
--- 427.85163927078247 seconds ---
Current file : Video_Set_1/ID_02/Camera_1/Seq_3.avi
--- 470.2572796344757 seconds ---
Current file : Video_Set_1/ID_02/Camera_2/Seq_1.avi
--- 482.2230417728424 seconds ---
Current file : Video_Set_1/ID_02/Camera_2/Seq_2

--- 6951.641211748123 seconds ---
Current file : Video_Set_1/ID_16/Camera_1/Seq_1.avi
--- 6978.401442050934 seconds ---
Current file : Video_Set_1/ID_16/Camera_1/Seq_2.avi
--- 7005.426823854446 seconds ---
Current file : Video_Set_1/ID_16/Camera_1/Seq_3.avi
--- 7111.232598543167 seconds ---
Current file : Video_Set_1/ID_16/Camera_2/Seq_1.avi
--- 7131.761817455292 seconds ---
Current file : Video_Set_1/ID_16/Camera_2/Seq_2.avi
--- 7145.166836977005 seconds ---
Current file : Video_Set_1/ID_16/Camera_2/Seq_3.avi
--- 7168.0930869579315 seconds ---
Current file : Video_Set_1/ID_16/Camera_2/Seq_4.avi
--- 7193.155578613281 seconds ---
Current file : Video_Set_1/ID_16/Camera_4/Seq_1.avi
--- 7242.393030405045 seconds ---
Current file : Video_Set_1/ID_16/Camera_4/Seq_2.avi
--- 7312.100495100021 seconds ---
Current file : Video_Set_1/ID_17/Camera_1/Seq_1.avi
--- 7711.41121172905 seconds ---
Current file : Video_Set_1/ID_17/Camera_4/Seq_1.avi
--- 8000.168614387512 seconds ---
Current file : Video